In [ ]:
%pip install --upgrade --no-cache-dir pip
%pip install --upgrade --no-cache-dir terra-notebook-utils

import os
import io
import pandas as pd
import firecloud.api as fapi

ws_namespace = os.environ['WORKSPACE_NAMESPACE']
ws_name = os.path.basename(os.path.dirname(os.getcwd()))
ws_bucket = os.environ['WORKSPACE_BUCKET']

In [ ]:
decryption_key = ws_bucket + "/prj_14641_D17100.ngc"

download_cols = ["entity:download_id", "request_num", "downloader", "token_string", "decryption_key"]
download_tbl = pd.DataFrame.from_records([
    ("MESA_LP4", "90865", "kwesterman", "A75497C4306B34AD49542C4D86C38B3B47C53EA9753904433B1AD0A16416865C86CA4C2A48F0E4F058C954C963AF54038D", decryption_key)
], columns = download_cols)

download_tbl_str = download_tbl.to_csv(sep = "\t", index = False)
fapi.upload_entities(ws_namespace, ws_name, download_tbl_str, "flexible")

In [ ]:
download_tbl

In [ ]:
downloaded_tsv = fapi.get_entities_tsv(ws_namespace, ws_name, "download", model="flexible")
downloaded_df = (pd.read_csv(io.StringIO(downloaded_tsv.text), sep='\t'))
#            .query("callset == 'Freeze 8' & data_type == 'Variant Call'")
#            .filter(["file_name", "project_id", "object_id"])
#            .assign(chr = lambda x: x.file_name.str.extract(r'chr([0-9X]*)')))

In [ ]:
downloaded_df

In [ ]:
for r in ["MESA_LP4"]:
    decrypted_files = downloaded_df.loc[downloaded_df["entity:download_id"] == r, "decrypted_files"]
    fnames = decrypted_files.iloc[0].strip("[]").split(",")
    for f in fnames:
        f = f.strip("\"")
        ! gsutil cp $f gs://fc-c5011c7b-5cea-4987-8069-216d079d539f/phenotypes/